In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.signal import argrelextrema

In [2]:
contacts_df = pd.read_excel('/content/Contacts_clear.xlsx', dtype={'Id': str})
calls_df = pd.read_excel('/content/Calls_clear.xlsx', dtype={'Id': str, "CONTACTID": str})
spend_df = pd.read_excel('/content/Spend_clear.xlsx')
deals_df = pd.read_excel('/content/Deals_clear.xlsx', dtype={'Id': str, 'Contact Name': str})

# Анализ временных рядов:


# Проанализируйте тенденцию создания сделок с течением времени и их связь с звонками.

In [20]:
contacts_df['Created Date'] = contacts_df['Created Time'].dt.date
calls_df['Call Date'] = calls_df['Call Start Time'].dt.date
deals_df['Deal Created Date'] = deals_df['Created Time'].dt.date

deals_per_day = deals_df.groupby('Deal Created Date').size().reset_index(name='Deals Count')
calls_per_day = calls_df.groupby('Call Date').size().reset_index(name='Calls Count')

time_series_df = pd.merge(deals_per_day, calls_per_day, left_on='Deal Created Date', right_on='Call Date', how='outer').dropna()
time_series_df.rename(columns={'Deal Created Date': 'Date'}, inplace=True)

In [21]:
fig = go.Figure()
time_series_df['Date'] = pd.to_datetime(time_series_df['Date'])
fig.add_trace(go.Scatter(x=time_series_df['Date'], y=time_series_df['Deals Count'],
                         mode='lines', name='Deals Count'))

deals_trend = px.scatter(time_series_df, x='Date', y='Deals Count', trendline='ols', template='plotly_white')
trend_data = deals_trend.data[1]
fig.add_trace(trend_data)

fig.add_trace(go.Scatter(x=time_series_df['Date'], y=time_series_df['Calls Count'],
                         mode='lines', name='Calls Count'))

calls_trend = px.scatter(time_series_df, x='Date', y='Calls Count', trendline='ols', template='plotly_white')
trend_data = calls_trend.data[1]
fig.add_trace(trend_data)

fig.update_layout(
    title='Trends of Deals and Calls Over Time with Trend Lines',
    xaxis_title='Date',
    yaxis_title='Count',
    legend_title='Legend',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)

fig.show()

In [22]:
numeric_time_series_df = time_series_df[['Date', 'Deals Count', 'Calls Count']]

weekly_time_series_df = numeric_time_series_df.resample('W', on='Date').sum().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'], y=weekly_time_series_df['Deals Count'],
                         mode='lines+markers', name='Deals Count (Weekly)'))

deals_trend = px.scatter(weekly_time_series_df, x='Date', y='Deals Count', trendline='ols', template='plotly_white')
trend_data = deals_trend.data[1]
fig.add_trace(trend_data)


fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'], y=weekly_time_series_df['Calls Count'],
                         mode='lines+markers', name='Calls Count (Weekly)'))

calls_trend = px.scatter(weekly_time_series_df, x='Date', y='Calls Count', trendline='ols', template='plotly_white')
trend_data = calls_trend.data[1]
fig.add_trace(trend_data)

fig.update_layout(
    title='Weekly Trends of Deals and Calls Over Time with Trend Lines',
    xaxis_title='Date',
    yaxis_title='Count',
    legend_title='Legend',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)

fig.show()

In [23]:
contacts_per_day = contacts_df.groupby('Created Date').size().reset_index(name='Contacts Count')

contacts_per_day['Created Date'] = pd.to_datetime(contacts_per_day['Created Date'])

spend_per_day = spend_df.groupby('Date').agg({'Spend': 'sum', 'Impressions': 'sum', 'Clicks': 'sum'}).reset_index()

print(time_series_df.dtypes)
print(spend_per_day.dtypes)

time_series_df['Date'] = pd.to_datetime(time_series_df['Date'])
spend_per_day['Date'] = pd.to_datetime(spend_per_day['Date'])

Date           datetime64[ns]
Deals Count           float64
Call Date              object
Calls Count           float64
dtype: object
Date           datetime64[ns]
Spend                 float64
Impressions             int64
Clicks                  int64
dtype: object


In [24]:
time_series_df = pd.merge(time_series_df, contacts_per_day, left_on='Date', right_on='Created Date', how='outer').dropna()
time_series_df = pd.merge(time_series_df, spend_per_day, on='Date', how='outer').dropna()

time_series_df.drop(columns=['Created Date'], inplace=True)

In [25]:
time_series_df['Date'] = pd.to_datetime(time_series_df['Date'])

numeric_columns = time_series_df.select_dtypes(include=[np.number]).columns
weekly_time_series_df = time_series_df.set_index('Date').resample('W')[numeric_columns].sum().reset_index()

In [26]:
fig = go.Figure()


fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'], y=weekly_time_series_df['Deals Count'],
                         mode='lines+markers', name='Deals Count (Weekly)'))

fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'], y=weekly_time_series_df['Calls Count'],
                         mode='lines+markers', name='Calls Count (Weekly)'))

fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'], y=weekly_time_series_df['Contacts Count'],
                         mode='lines+markers', name='Contacts Count (Weekly)', line=dict(dash='dash')))


fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'], y=weekly_time_series_df['Spend'],
                         mode='lines+markers', name='Ad Spend (Weekly)', line=dict(dash='dot')))

fig.update_layout(
    title='Weekly Trends of Deals, Calls, Contacts, and Ad Spend',
    xaxis_title='Date',
    yaxis_title='Count / Spend',
    legend_title='Legend',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)
fig.show()

## Вывод: Графики демонстрируют постепенное увеличение количества сделок и звонков с течением времени. Заметны колебания, но оба показателя показывают общий восходящий тренд. Это указывает на связь между ростом количества звонков и увеличением числа сделок, подтверждая, что более активное взаимодействие с клиентами через звонки может способствовать увеличению количества сделок.Включённые трендовые линии подтверждают общую тенденцию роста как для звонков, так и для сделок.

# Изучите распределение времени закрытия сделок и продолжительность периода от создания до закрытия.






In [27]:
fig = px.histogram(
    deals_df,
    x='Difference in Days',
    title='Распределение продолжительности сделок',
    labels={'Difference in Days': 'Продолжительность (в днях)'},
    template='plotly_white',
    nbins=20
)
fig.update_layout(
    xaxis_title='Продолжительность (в днях)',
    yaxis_title='Количество сделок',
    title=dict(x=0.5)
)
fig.show()

In [28]:
weekly_closing_distribution = deals_df.resample('W', on='Closing Date').size().reset_index(name='Deals Closed')
weekly_closing_distribution = weekly_closing_distribution[weekly_closing_distribution['Deals Closed'] >= 5]


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=weekly_closing_distribution['Closing Date'],
    y=weekly_closing_distribution['Deals Closed'],
    mode='lines+markers',
    name='Deals Closed (Weekly)'
))
fig.update_layout(
    title=dict(
        text='Распределение сделок по дате закрытия',
        x=0.5
    ),
    xaxis_title='Дата закрытия',
    yaxis_title='Количество закрытых сделок',
    template='plotly_white',
    xaxis=dict(tickangle=45)
)

fig.show()

In [29]:
deals_df['Time_to_Close'] = abs(deals_df['Closing Date'] - deals_df['Created Time']).dt.days

successful_deals_time = deals_df[deals_df['Stage'] == 'Payment Done']['Time_to_Close'].dropna()
lost_deals_time = deals_df[deals_df['Stage'] == 'Lost']['Time_to_Close'].dropna()

hist_data = [successful_deals_time, lost_deals_time]
group_labels = ['Успешные сделки', 'Потерянные сделки']

fig = ff.create_distplot(
    hist_data,
    group_labels,
    show_hist=True,
    show_rug=False,
    bin_size=5,
    colors=['blue', 'red']
)

fig.update_layout(
    title=dict(text="Распределение времени закрытия сделок", x=0.45),
    xaxis_title="Время закрытия сделок (в днях)",
    yaxis_title="Плотность сделок",
    bargap=0.05
)

fig.show()

## Вывод: Большинство сделок закрываются в первые недели после создания, и более продолжительные взаимодействия чаще приводят к успеху. Успешные сделки занимают больше времени на закрытие по сравнению с потерянными, что может указывать на необходимость длительного взаимодействия с клиентом для достижения результата

# Анализ эффективности компаний:

# Сравните эффективность различных кампаний с точки зрения генерации лидов и коэффициента конверсии.


In [3]:
display(deals_df['Stage'].value_counts())

Stage
Lost                         13890
Call Delayed                  2163
Registered on Webinar         2065
Payment Done                   843
Waiting For Payment            323
Qualificated                   128
Registered on Offline Day       85
Need to Call - Sales            32
Need To Call                    31
Test Sent                       25
Need a consultation             23
New Lead                         5
Free Education                   1
Name: count, dtype: int64

In [4]:
contacts_and_deals = pd.merge(
    contacts_df,
    deals_df[['Campaign', 'Contact Name', 'Stage', 'Id']],
    left_on='Id',
    right_on='Contact Name',
    how='inner'
)

campaign_data = pd.merge(
    contacts_and_deals,
    spend_df[['Campaign', 'Impressions', 'Clicks', 'Spend']],
    on='Campaign',
    how='left'
)

leads_per_campaign = campaign_data.groupby('Campaign').size().reset_index(name='Leads')

closed_deals_per_campaign = campaign_data[campaign_data['Stage'] == 'Payment Done'].groupby('Campaign').size().reset_index(name='Closed Deals')

campaign_analysis = pd.merge(
    leads_per_campaign,
    closed_deals_per_campaign,
    on='Campaign',
    how='left'
)

campaign_analysis['Conversion Rate (%)'] = (campaign_analysis['Closed Deals'] / campaign_analysis['Leads']) * 100

campaign_analysis.fillna(0, inplace=True)


display(campaign_analysis)

Campaign    Leads  Closed Deals  Conversion Rate (%)
0            01.04.23women_PL     3131           0.0             0.000000
1             02.07.23wide_DE  1512460       83668.0             5.531915
2        02.08.23interests_DE      288           0.0             0.000000
3               03.07.23women   655095       34131.0             5.210084
4    04.07.23recentlymoved_DE   942456       39804.0             4.223433
..                        ...      ...           ...                  ...
146                   work_wr        6           0.0             0.000000
147              workingin_DE       48           0.0             0.000000
148                     yo_DE       13           0.0             0.000000
149         youtube_shorts_DE  1945524       64607.0             3.320802
150       youtube_shortsin_AT     1862           0.0             0.000000

[151 rows x 4 columns]

In [5]:
lost_analysis = campaign_data[campaign_data['Stage'] == 'Lost'].groupby('Campaign').size().reset_index(name='Lost Deals')
display(lost_analysis)

Campaign  Lost Deals
0            01.04.23women_PL        2727
1             02.07.23wide_DE     1200314
2        02.08.23interests_DE         288
3               03.07.23women      504258
4    04.07.23recentlymoved_DE      746004
..                        ...         ...
139                   work_wr           6
140              workingin_DE          43
141                     yo_DE          11
142         youtube_shorts_DE     1563977
143       youtube_shortsin_AT        1729

[144 rows x 2 columns]

In [6]:
threshold = 5
filtered_campaign_analysis = campaign_analysis[campaign_analysis['Conversion Rate (%)'] >= threshold]
filtered_campaign_analysis = filtered_campaign_analysis.sort_values(by='Conversion Rate (%)', ascending=False)

fig = px.bar(
    filtered_campaign_analysis,
    x='Campaign',
    y='Conversion Rate (%)',
    title='Коэффициент конверсии по кампаниям',
    labels={'Conversion Rate (%)': 'Коэффициент конверсии (%)', 'Campaign': 'Кампания'},
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='Кампания',
    yaxis_title='Коэффициент конверсии (%)',
    title=dict(text='Коэффициент конверсии по кампаниям выше 5%', x=0.5)
)
fig.show()

In [7]:
filtered_lost_analysis = lost_analysis[lost_analysis['Campaign'] != 'Unknown']
threshold = 25000
filtered_lost_analysis = filtered_lost_analysis[filtered_lost_analysis['Lost Deals'] > threshold]
sorted_filtered_lost_analysis = filtered_lost_analysis.sort_values(by='Lost Deals', ascending=False)

fig = px.bar(
    sorted_filtered_lost_analysis,
    x='Campaign',
    y='Lost Deals',
    title='Топ-10 кампаний с потерянными сделками',
    labels={'Lost Deals': 'Количество потерянных сделок', 'Campaign': 'Кампания'},
    template='plotly_white'
)
fig.update_layout(
    xaxis_title='Кампания',
    yaxis_title='Количество потерянных сделок',
    title=dict(text='Топ-10 кампаний с потерянными сделками', x=0.5)
)
fig.show()

In [16]:
filtered_campaign_analysis = campaign_analysis[
    (campaign_analysis['Leads'] > 100) &
    (campaign_analysis['Campaign'] != 'Unknown')
].sort_values(by='Leads', ascending=False)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=filtered_campaign_analysis['Campaign'],
    y=filtered_campaign_analysis['Leads'],
    name='Количество лидов',
    marker_color='blue',
    yaxis='y1'
))

fig.add_trace(go.Bar(
    x=filtered_campaign_analysis['Campaign'],
    y=filtered_campaign_analysis['Closed Deals'],
    name='Успешные сделки',
    marker_color='green',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=filtered_campaign_analysis['Campaign'],
    y=filtered_campaign_analysis['Conversion Rate (%)'],
    mode='lines+markers',
    name='Коэффициент конверсии (%)',
    marker=dict(color='red'),
    yaxis='y2'
))

fig.update_layout(
    title=dict(text='Эффективность кампаний: генерация лидов и конверсия', x=0.45),
    xaxis=dict(title='Кампания'),
    yaxis=dict(
        title='Количество сделок',
        titlefont=dict(color='blue'),
        side='left'
    ),
    yaxis2=dict(
        title='Конверсия',
        titlefont=dict(color='red'),
        overlaying='y',
        side='right',
        showgrid=False,
        title_standoff=20
    ),
    margin=dict(l=70, r=90, t=70, b=70),
    barmode='group',
    template='plotly_white'
)

fig.show()

## Вывод: Кампании с большим количеством лидов не всегда имеют высокий коэффициент конверсии, что говорит о разных подходах к управлению лидами. Кампании с высоким количеством успешных сделок демонстрируют лучшую эффективность, но их коэффициент конверсии всё же варьируется.

# Оцените эффективность различных маркетинговых источников (Source) в генерировании качественных лидов.

In [28]:
display(deals_df['Source'].value_counts())
display(deals_df['Quality'].value_counts())

Source
Facebook Ads      4727
Google Ads        4113
Tiktok Ads        2003
SMM               1668
Youtube Ads       1618
Organic           1469
CRM               1454
Bloggers          1073
Telegram posts     993
Webinar            305
Partnership        189
Offline              2
Name: count, dtype: int64

Quality
E - Non Qualified    6100
D - Non Target       5972
C - Low              3373
Unknown              2230
B - Medium           1525
A - High              414
Name: count, dtype: int64

In [29]:
contacts_deals = pd.merge(
    contacts_df,
    deals_df[['Contact Name', 'Source', 'Quality']],
    left_on='Id',
    right_on='Contact Name',
    how='inner'
)
source_analysis = contacts_deals.groupby('Source').agg(
    Leads=('Id', 'count'),
    QualityLeads=('Quality', lambda x: (x == 'A - High Potential').sum() + (x == 'B - Medium').sum())
).reset_index()

source_analysis['Quality Rate (%)'] = (source_analysis['QualityLeads'] / source_analysis['Leads']) * 100

display(source_analysis)

Source  Leads  QualityLeads  Quality Rate (%)
0         Bloggers   1070            83          7.757009
1              CRM   1454            30          2.063274
2     Facebook Ads   4725           462          9.777778
3       Google Ads   4105           311          7.576127
4          Offline      2             0          0.000000
5          Organic   1469           199         13.546630
6      Partnership    189             2          1.058201
7              SMM   1665           124          7.447447
8   Telegram posts    986            92          9.330629
9       Tiktok Ads   1998            88          4.404404
10         Webinar    305            33         10.819672
11     Youtube Ads   1618           100          6.180470

In [30]:
source_spend_analysis = pd.merge(
    source_analysis,
    spend_df[['Source', 'Spend']],
    on='Source',
    how='left'
)
source_spend_analysis['Cost per Quality Lead'] = source_spend_analysis['Spend'] / source_spend_analysis['QualityLeads']
display(source_spend_analysis)

Source  Leads  QualityLeads  Quality Rate (%)   Spend  \
0         Bloggers   1070            83          7.757009    0.00   
1         Bloggers   1070            83          7.757009   25.00   
2         Bloggers   1070            83          7.757009    0.00   
3         Bloggers   1070            83          7.757009  150.00   
4         Bloggers   1070            83          7.757009    0.00   
...            ...    ...           ...               ...     ...   
14880  Youtube Ads   1618           100          6.180470    0.21   
14881  Youtube Ads   1618           100          6.180470    0.01   
14882  Youtube Ads   1618           100          6.180470    0.00   
14883  Youtube Ads   1618           100          6.180470    0.24   
14884  Youtube Ads   1618           100          6.180470    0.51   

       Cost per Quality Lead  
0                   0.000000  
1                   0.301205  
2                   0.000000  
3                   1.807229  
4                   0.000000  
...                      ...  
14880               0.002100  
14881               0.000100  
14882               0.000000  
14883               0.002400  
14884               0.005100  

[14885 rows x 6 columns]

In [31]:
sorted_source_analysis = source_analysis.sort_values(by='Leads', ascending=False)

fig = px.bar(
    sorted_source_analysis,
    x='Source',
    y='Leads',
    title='Количество лидов по источникам',
    labels={'Leads': 'Лиды', 'Source': 'Источник'},
    template='plotly_white'
)
fig.update_layout(
    title=dict(text='Количество лидов по источникам', x=0.5)
)

fig.show()

In [32]:
sorted_source_analysis = source_analysis.sort_values(by='Quality Rate (%)', ascending=False)


fig = px.bar(
    sorted_source_analysis,
    x='Source',
    y='Quality Rate (%)',
    title='Коэффициент качества лидов по источникам',
    labels={'Quality Rate (%)': 'Коэффициент качества (%)', 'Source': 'Источник'},
    template='plotly_white'
)

fig.update_layout(
    title=dict(text='Коэффициент качества лидов по источникам', x=0.5)
)

fig.show()

In [33]:
source_analysis = pd.merge(
    deals_df[['Source', 'Stage', 'Quality', 'Id']],
    spend_df[['Source', 'Impressions', 'Clicks', 'Spend']],
    on='Source',
    how='left'
)

leads_per_source = source_analysis.groupby('Source').agg(
    Total_Leads=('Id', 'count'),
    High_Quality_Leads=('Quality', lambda x: (x == 'A - High').sum()),
    Successful_Deals=('Stage', lambda x: (x == 'Payment Done').sum()),
    Total_Impressions=('Impressions', 'sum'),
    Total_Clicks=('Clicks', 'sum'),
    Total_Spend=('Spend', 'sum')
).reset_index()

leads_per_source['Conversion Rate (%)'] = (
    leads_per_source['Successful_Deals'] / leads_per_source['Total_Leads'] * 100
)
leads_per_source['Cost per Lead'] = leads_per_source['Total_Spend'] / leads_per_source['Total_Leads']

leads_per_source.fillna(0, inplace=True)

leads_per_source.sort_values('High_Quality_Leads', ascending=False, inplace=True)

display(leads_per_source)

Source  Total_Leads  High_Quality_Leads  Successful_Deals  \
2     Facebook Ads     41356523              953641           1758549   
3       Google Ads      3751056               84816            156864   
9       Tiktok Ads      4490726               73986            125552   
11     Youtube Ads      2304032               37024             75472   
5          Organic       339339               12705             32571   
8   Telegram posts       465717                7504             18760   
7              SMM       293568                6512             15840   
0         Bloggers       277907                5439             10101   
10         Webinar        97905                4815              8346   
1              CRM       114866                 711              1817   
6      Partnership         2835                   0                30   
4          Offline           16                   0                 0   

    Total_Impressions  Total_Clicks   Total_Spend  Conversion Rate (%)  \
2         13467100098     227510510  1.595041e+08             4.252168   
3        134710349742    1022027031  2.377256e+08             4.181862   
9         10029445636      56620804  2.400730e+07             2.795806   
11        14005302830      95560698  2.367642e+07             3.275649   
5                   0      50733384  0.000000e+00             9.598366   
8           692490396      16390458  6.782796e+06             4.028197   
7            39651696       9902916  1.212556e+07             5.395683   
0           789137850      15281666  1.442005e+07             3.634669   
10           92004470        988505  8.765517e+05             8.524590   
1                   0       5603716  0.000000e+00             1.581843   
6                   0         24759  0.000000e+00             1.058201   
4                   0            78  0.000000e+00             0.000000   

    Cost per Lead  
2        3.856805  
3       63.375658  
9        5.345972  
11      10.276081  
5        0.000000  
8       14.564200  
7       41.304091  
0       51.888031  
10       8.953084  
1        0.000000  
6        0.000000  
4        0.000000

In [34]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=leads_per_source['Source'],
    y=leads_per_source['Total_Leads'],
    name='Количество лидов',
    marker_color='blue',
    yaxis='y1'
))

fig.add_trace(go.Bar(
    x=leads_per_source['Source'],
    y=leads_per_source['High_Quality_Leads'],
    name='Качественные лиды',
    marker_color='green',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=leads_per_source['Source'],
    y=leads_per_source['Conversion Rate (%)'],
    mode='lines+markers',
    name='Коэффициент конверсии (%)',
    marker=dict(color='red'),
    yaxis='y2'
))

fig.update_layout(
    title=dict(text='Эффективность источников: лиды и конверсия', x=0.45),
    xaxis=dict(title='Источник', tickangle=-45),
    yaxis=dict(
        title='Количество лидов',
        titlefont=dict(color='blue'),
        side='left'
    ),
    yaxis2=dict(
        title='Конверсия',
        titlefont=dict(color='red'),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    margin=dict(l=70, r=90, t=70, b=70),
    barmode='group',
    legend=dict(title='Показатели', x=1.05, y=1)
)
fig.update_traces(
    marker=dict(line=dict(width=2, color='green')),
    selector=dict(name='Качественные лиды')
)
fig.show()

## Вывод: Приоритет следует отдавать источникам с высокой конверсией Organic, а источники с большим объёмом лидов, но низкой конверсией Facebook Ads, требуют оптимизации анализа. Это поможет сократить затраты на нерелевантных клиентов и повысить общую эффективность маркетинговой стратегии

# Анализ эффективности работы отдела продаж

# Оцените эффективность отдельных владельцев сделок и рекламных кампаний с точки зрения количества обработанных сделок, коэффициента конверсии и общей суммы продаж.


In [17]:
lost_reason_analysis = deals_df[deals_df['Stage'] == 'Lost'].groupby('Lost Reason').size().reset_index(name='Count')
sorted_lost_reason_analysis = lost_reason_analysis.sort_values(by='Count', ascending=False)
fig = px.bar(
    sorted_lost_reason_analysis,
    x='Count',
    y='Lost Reason',
    orientation='h',
    title='Причины потерь сделок (горизонтальная гистограмма с насыщенными цветами)',
    labels={'Lost Reason': 'Причина', 'Count': 'Количество'},
    template='plotly_white',
    color='Count',
    color_continuous_scale=['mediumseagreen', 'goldenrod', 'tomato']
)
fig.update_layout(
    title=dict(text='Причины потерь сделок', x=0.5),
    xaxis=dict(title='Количество'),
    yaxis=dict(title='Причина'),
    coloraxis_colorbar=dict(
        title='Количество',
        ticks='outside'
    )
)
fig.show()

## Вывод: График показывает, что наиболее распространённой причиной отказа или отсутствия ответа является отсутствие обратной связи от клиента "Не отвечает".Наименее частые причины, такие как "Беженец" и "Перешёл к конкурентам", встречаются крайне редко.

In [18]:
lost_owner_analysis = deals_df[deals_df['Stage'] == 'Lost'].groupby('Deal Owner Name').size().reset_index(name='Lost Deals')
lost_owner_analysis_sorted = lost_owner_analysis.sort_values(by='Lost Deals', ascending=False)
fig = px.area(
    lost_owner_analysis_sorted,
    x='Deal Owner Name',
    y='Lost Deals',
    title='Потери по владельцам сделок',
    labels={'Deal Owner Name': 'Владелец сделки', 'Lost Deals': 'Потери'},
    template='plotly_white',
)
fig.update_layout(title=dict(text='Потери по владельцам сделок', x=0.5), xaxis=dict(tickangle=-45))
fig.show()

## Вывод: График показывает, что самые большие потери среди владельцев сделок наблюдаются у Charlie Davis, тогда как у Wendy Clark они минимальны. Это позволяет идентифицировать зоны для улучшения в эффективности работы каждого владельца.

In [19]:
owner_reason_analysis = deals_df[deals_df['Stage'] == 'Lost'].groupby(['Deal Owner Name', 'Lost Reason']).size().reset_index(name='Lost Count')
owner_reason_analysis_sorted = owner_reason_analysis.sort_values(by='Lost Count', ascending=False)
fig = px.bar(
    owner_reason_analysis_sorted,
    x='Deal Owner Name',
    y='Lost Count',
    color='Lost Reason',
    title='Причины потерь по владельцам сделок',
    labels={'Deal Owner Name': 'Владелец сделки', 'Lost Count': 'Количество потерь', 'Lost Reason': 'Причина потери'},
    template='plotly_white'
)

fig.update_layout(title=dict(text='Причины потерь по владельцам сделок', x=0.5), xaxis=dict(tickangle=-45))
fig.show()

## Вывод: График показывает, что Charlie Davis лидирует по количеству потерь сделок, с основной причиной "Не отвечает", за которой следуют другие причины, такие как "Перестал отвечать" и "Изменил решение". Это может указывать на необходимость улучшения взаимодействия с клиентами.

In [20]:
fig = px.bar(
    owner_reason_analysis_sorted[owner_reason_analysis_sorted['Lost Count'] > 100],
    x='Deal Owner Name',
    y='Lost Count',
    color='Lost Reason',
    title='Причины потерь у крупных владельцев',
    labels={'Deal Owner Name': 'Владелец сделки', 'Lost Count': 'Количество потерь', 'Lost Reason': 'Причина'},
    template='plotly_white'
)
fig.update_layout(title=dict(text='Причины потерь у крупных владельцев', x=0.5), xaxis=dict(tickangle=-45))
fig.show()

In [21]:
deals_df['Created Time'] = pd.to_datetime(deals_df['Created Time'], errors='coerce')
deals_df['Closing Date'] = pd.to_datetime(deals_df['Closing Date'], errors='coerce')


successful_deals = deals_df[deals_df['Stage'] == 'Payment Done']

deal_owner_detailed_analysis = deals_df.groupby('Deal Owner Name').agg(
    Total_Deals=('Stage', 'count'),
    Converted_Deals=('Stage', lambda x: (x == 'Payment Done').sum()),
    Conversion_Rate=('Stage', lambda x: (x == 'Payment Done').sum() / len(x) * 100 if len(x) > 0 else 0),
    Total_Sales=('Initial Amount Paid', 'sum'),
    Avg_Deal_Amount=('Initial Amount Paid', lambda x: x[x > 0].mean()),
    Avg_Time_to_Close=('Closing Date', lambda x: (x - deals_df.loc[x.index, 'Created Time']).dt.days.mean())
).reset_index()

campaign_detailed_analysis = deals_df.groupby('Campaign').agg(
    Total_Deals=('Stage', 'count'),
    Converted_Deals=('Stage', lambda x: (x == 'Payment Done').sum()),
    Conversion_Rate=('Stage', lambda x: (x == 'Payment Done').sum() / len(x) * 100 if len(x) > 0 else 0),
    Total_Sales=('Initial Amount Paid', 'sum'),
    Avg_Deal_Amount=('Initial Amount Paid', lambda x: x[x > 0].mean()),
    Avg_Time_to_Close=('Closing Date', lambda x: (x - deals_df.loc[x.index, 'Created Time']).dt.days.mean())
).reset_index()


stage_distribution_owner = deals_df.pivot_table(
    index='Deal Owner Name', columns='Stage', values='Id', aggfunc='count', fill_value=0
).reset_index()

stage_distribution_campaign = deals_df.pivot_table(
    index='Campaign', columns='Stage', values='Id', aggfunc='count', fill_value=0
).reset_index()

In [22]:
fig1 = go.Figure()

fig1.add_trace(go.Bar(
    x=deal_owner_detailed_analysis['Deal Owner Name'],
    y=deal_owner_detailed_analysis['Conversion_Rate'],
    name='Конверсия (%)',
    marker_color='blue',
    yaxis="y1"
))

fig1.add_trace(go.Scatter(
    x=deal_owner_detailed_analysis['Deal Owner Name'],
    y=deal_owner_detailed_analysis['Total_Sales'],
    name='Общий объём продаж',
    mode='lines+markers',
    marker=dict(color='red'),
    yaxis="y2"
))

fig1.update_layout(
    title="Конверсия и общий объём продаж по владельцам сделок",
    xaxis_title="Владелец сделки",
    yaxis=dict(title='Конверсия (%)', side="left"),
    yaxis2=dict(title='Общий объём продаж', overlaying="y", side="right", showgrid=False),
    barmode='group'
)

fig1.show()

## Вывод: Oliver Taylor имеет высокую конверсия и низкий уровень продаж это может свидетельствовать о том, что он работает преимущественно с небольшими сделками, тогда как другие, например, Cara Iverson, достигают высоких показателей продаж при более низкой конверсии. Это подчёркивает различные подходы владельцев сделок к управлению процессами.

In [23]:
fig2 = px.bar(campaign_detailed_analysis, x='Campaign', y='Avg_Deal_Amount',
              title="Средняя сумма сделки по кампаниям", labels={'Avg_Deal_Amount': "Средняя сумма сделки"},
              color='Avg_Deal_Amount', color_continuous_scale='Teal')
fig2.update_layout(
    title=dict(text="Средняя сумма сделки по кампаниям", x=0.5),
    xaxis_title="Кампания",
    yaxis_title="Средняя сумма сделки"
)

fig2.show()

## Вывод: График показывает среднюю сумму сделки по различным кампаниям. Наиболее высокие показатели наблюдаются у кампаний "Relocat_DE" "Berlin_DE," и "Aussiedler_DE," что может свидетельствовать о их высокой эффективности. Остальные кампании имеют сравнительно меньшую среднюю сумму сделки.

In [24]:
stage_distribution_owner.reset_index(inplace=True)
fig3 = px.bar(stage_distribution_owner, x='Deal Owner Name', y=stage_distribution_owner.columns[1:],
              title="Распределение этапов по владельцам сделок", labels={'value': 'Количество сделок', 'variable': 'Этап'},
              color_discrete_sequence=px.colors.qualitative.Prism)
fig3.update_layout(
    title=dict(text="Распределение этапов по владельцам сделок", x=0.5),
    xaxis_title="Владелец сделки",
    yaxis_title="Количество сделок",
    barmode='stack'
)
fig3.show()

## Вывод: График показывает распределение этапов сделок по владельцам. Наибольшее количество сделок у Charlie Davis, за ним следуют Julia Nelson и Ulysses Adams.У  Charlie Davis самое большее количество в стадии Lost-1954 и в стадии Payment Done-145 сделок. Этапы сделок распределены неравномерно, что указывает на различия в подходах и результативности работы владельцев.

In [25]:
stage_distribution_campaign.reset_index(inplace=True)
fig4 = px.bar(stage_distribution_campaign, x='Campaign', y=stage_distribution_campaign.columns[1:],
              title="Распределение этапов по кампаниям", labels={'value': 'Количество сделок', 'variable': 'Этап'},
              color_discrete_sequence=px.colors.qualitative.Vivid)
fig4.update_layout(
    title=dict(text="Распределение этапов по кампаниям", x=0.5),
    xaxis_title="Кампания",
    yaxis_title="Количество сделок",
    barmode='stack'
)

fig4.show()

## Вывод: График показывает распределение этапов сделок по различным кампаниям. Наиболее активной кампанией является "Trigger_DE", с наибольшим количеством сделок, большинство из которых находятся на этапах "New Lead" и "Payment Done". Другие кампании, такие как "Markt_DE" и "Tristar_DE", также демонстрируют значительные показатели, но с меньшим объемом сделок. Это позволяет оценить эффективность каждой кампании по этапам сделок и выделить наиболее успешные стратегии.

# Анализ платежей и продуктов

# Изучите распределение типов оплаты и их влияние на успешность сделок.

In [35]:
successful_deals = deals_df[deals_df['Stage'] == 'Payment Done']

payment_distribution = deals_df['Payment Type'].value_counts().reset_index()
payment_distribution.columns = ['Payment Type', 'Count']

successful_payment_distribution = successful_deals['Payment Type'].value_counts().reset_index()
successful_payment_distribution.columns = ['Payment Type', 'Count']

payment_success_analysis = pd.merge(payment_distribution, successful_payment_distribution, on='Payment Type', how='left')
payment_success_analysis.columns = ['Payment Type', 'Total Deals', 'Successful Deals']
payment_success_analysis['Success Rate (%)'] = (payment_success_analysis['Successful Deals'] / payment_success_analysis['Total Deals']) * 100

display(payment_success_analysis)

Payment Type  Total Deals  Successful Deals  Success Rate (%)
0  Recurring Payments          337               245         72.700297
1         One Payment          135               111         82.222222
2         Reservation            5                 1         20.000000

In [36]:
fig = px.pie(
    payment_success_analysis,
    names='Payment Type',
    values='Total Deals',
    title='Распределение типов оплаты',
    color_discrete_sequence=['teal', 'coral', 'gold']
)
fig.update_layout(
    title=dict(text='Распределение типов оплаты', x=0.5),
    legend=dict(title='Типы оплаты')
)

fig.show()

## Вывод:Основная доля приходится на периодические платежи (70.6%), что свидетельствует о преобладании подписок или регулярных платежей. Единичные платежи занимают 28.3%, а наименьшую долю составляют резервации (1.05%). Это указывает на сильный акцент на долгосрочных финансовых соглашениях.

In [37]:
fig = px.pie(
    payment_success_analysis,
    names='Payment Type',
    values='Success Rate (%)',
    title='Успешность типов оплаты',
    hole=0.5,
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(
    title=dict(text='Успешность типов оплаты', x=0.45),
    legend=dict(title='Типы оплаты')
)
fig.show()

In [38]:
payment_analysis = deals_df.groupby('Payment Type').agg(
    Total_Deals=('Id', 'count'),
    Successful_Deals=('Stage', lambda x: (x == 'Payment Done').sum())
).reset_index()

payment_analysis['Success Rate (%)'] = (payment_analysis['Successful_Deals'] / payment_analysis['Total_Deals']) * 100

fig = px.bar(
    payment_analysis,
    x='Payment Type',
    y=['Total_Deals', 'Successful_Deals'],
    title='Распределение типов оплаты и их успешность',
    labels={'value': 'Количество сделок', 'Payment Type': 'Тип оплаты', 'variable': 'Метрика'},
    barmode='group',
    text_auto=True
)

fig.add_scatter(
    x=payment_analysis['Payment Type'],
    y=payment_analysis['Success Rate (%)'],
    mode='lines+markers',
    name='Коэффициент успешности (%)',
    marker=dict(color='red')
)

fig.update_layout(
    title=dict(text='Распределение типов оплаты и успешность сделок', x=0.45),
    yaxis=dict(title='Количество сделок'),
    yaxis2=dict(
        title='Коэффициент успешности (%)',
        overlaying='y',
        side='right'
    )
)
fig.show()

## Вывод: Recurring Payments (периодические платежи) имеют наибольшее количество успешных сделок, что говорит об их популярности и стабильности. Однако One Payment (единоразовые платежи) показывают более высокий коэффициент успешности, что делает их эффективными для быстрого достижения результата.

# Проанализируйте популярность и успешность различных продуктов и типов обучения

In [26]:
product_analysis = deals_df.groupby(['Product', 'Education Type']).agg(
    Total_Deals=('Id', 'count'),
    Successful_Deals=('Stage', lambda x: (x == 'Payment Done').sum()),
    Total_Revenue=('Offer Total Amount', 'sum')
).reset_index()

product_analysis['Success Rate (%)'] = (product_analysis['Successful_Deals'] / product_analysis['Total_Deals']) * 100

product_analysis['Success Rate (%)'] = product_analysis['Success Rate (%)'].round(2)

product_analysis.sort_values('Success Rate (%)', ascending=False, inplace=True)

display(product_analysis)

Product Education Type  Total_Deals  Successful_Deals  \
0  Digital Marketing        Evening          245               112   
2       UX/UI Design        Evening          150                57   
5      Web Developer        Morning          533               135   
1  Digital Marketing        Morning         1491               348   
3       UX/UI Design        Morning          787               168   
4      Web Developer        Evening            1                 0   

   Total_Revenue  Success Rate (%)  
0         807109             45.71  
2         519350             38.00  
5        2835800             25.33  
1       15457509             23.34  
3        8008900             21.35  
4           2000              0.00

In [27]:
fig = px.bar(
    product_analysis,
    x='Product',
    y='Success Rate (%)',
    color='Education Type',
    text='Success Rate (%)',
    title='Успешность продуктов и типов обучения',
    labels={'Product': 'Продукт', 'Success Rate (%)': 'Коэффициент успешности (%)', 'Education Type': 'Тип обучения'}
)

fig.update_traces(textposition='auto')
fig.update_layout(
    title=dict(
        text='Успешность продуктов и типов обучения',
        x=0.5
    ),
    xaxis=dict(tickangle=-45)
)
fig.show()

## Вывод: Продукты Digital Marketing и UX/UI Design показывают более высокие показатели успешности для вечернего типа обучения, в то время как утренние курсы имеют заметно меньшую успешность. Для продукта Web Developer данные есть только по утренним курсам, где успешность также ниже по сравнению с вечерними показателями других продуктов. Это подчеркивает популярность и эффективность вечернего обучения для большинства продуктов